In [2]:
import math

import pandas as pd

df = pd.read_pickle(r'data\data_df.pkl')
df.head()

,OriginalText,CleanedText,WordList,Sentyment,Lubienie,Wstręt,Złość,Strach,Zaskoczenie,Oczekiwanie,Radość,Smutek
AF1.txt,Wczoraj rano przytrafiło mi się coś miłego. Pr...,wczoraj rano przytrafiło mi się coś miłego prz...,"[wczoraj, rano, przytrafiło, mi, się, coś, mił...",5.0,2.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0
AF10.txt,"Pan Dussel, człowiek, o którym zawsze mówiono,...",pan dussel człowiek o którym zawsze mówiono że...,"[pan, dussel, człowiek, o, którym, zawsze, mów...",1.0,0.0,1.0,4.0,0.0,0.0,1.0,0.0,2.0
AF11.txt,Jakbym w ciągu dnia nie słyszała wystarczająco...,jakbym w ciągu dnia nie słyszała wystarczająco...,"[jakbym, w, ciągu, dnia, nie, słyszała, wystar...",1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,2.0
AF12.txt,"Pienię się z wściekłości, a nie wolno mi tego ...",pienię się z wściekłości a nie wolno mi tego o...,"[pienię, się, z, wściekłości, a, nie, wolno, m...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
AF13.txt,"Wszyscy uważają, że jestem przesadna, gdy mówi...",wszyscy uważają że jestem przesadna gdy mówię ...,"[wszyscy, uważają, że, jestem, przesadna, gdy,...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0


In [12]:
from sklearn.base import clone
import numpy as np
class OrdinalClassifier:

    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}

    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0] - 1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf

    def predict_proba(self, X):
        clfs_predict = {k: self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i, y in enumerate(self.unique_class):
            print(y)

            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:, 1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                predicted.append(clfs_predict[y - 1][:, 1] - clfs_predict[y][:, 1])
            else:
                # Vk = Pr(y > Vk-1)
                if y - 1 in clfs_predict.keys():
                    predicted.append(clfs_predict[y - 1][:, 1])
                else:
                    raise ValueError
        return np.vstack(predicted).T


    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)



from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
X = df['CleanedText']

ylabels = df['Sentyment']
ylabels = ylabels.apply(lambda x: x-1)# the labels, or answers, we want to test against
vector = CountVectorizer()
X_vec = vector.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X_vec, ylabels, test_size=0.3)

model = MultinomialNB()
clf = OrdinalClassifier(model)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)



0.0
1.0
2.0
3.0
4.0


In [7]:
print(y_pred)
print(y_test.values - y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
[ 0.  0.  0.  2.  0.  2.  0.  0.  0.  0.  2.  0.  2.  0.  4.  0.  3.  0.
  0.  2.  2.  0.  0.  0.  1.  0.  0.  1.  0.  0. -1.]
